# Sprint18

## GitHubのデータ適用
今回、参考にした「GitHub」からの「Unet」をkaggleのデータセットに合わせるにはフォルダーの構成を変更する必要がある。

## Unet Train

In [1]:
from model import *
from data import *

Using TensorFlow backend.


In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/kaggle/train','images','masks',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=1,epochs=1,callbacks=[model_checkpoint])

/home/hiro/workspase_python/diveintocode-term2/Sprint18/unet-master/model.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/1
Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
test
1/1 [==============================] - 21s 21s/step - loss: 0.7148 - acc: 0.4598

Epoch 00001: loss improved from inf to 0.71478, saving model to unet_membrane.hdf5


## Unet Predict

In [6]:
testGene = testGenerator("./data/kaggle/test/images")

model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,1,verbose=1)
saveResult(save_path="data/membrane/test",test_path="./data/kaggle/test/images",npyfile=results)

1/1 [==============================] - 3s 3s/step


## プレディク画像ファイル名を変更
入力画像と同じ名前で保存するには以下の様に修正する必要がある。

In [4]:
def saveResult(save_path,test_path,npyfile,flag_multi_class = False,num_class = 2):
    #for idx, img_name in enumerate(sorted(os.listdir(test_path))):
    testname = sorted(os.listdir(test_path))
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        name = testname[i]
        name = name.replace('.png', '')
        io.imsave(os.path.join(save_path,"%s_predict.png"%name),img)

## 論文とソースコード

今回のコア部分である「Unet」は「model.py」に実装されている。<br>
Dawn Sampling、Up Sampling ともに畳み込み後に活性化関数(ReLu)に掛けている。<br>
Down Smapling を5層まで行い、それ以降の層では Up Smapling を行っている。<br>

```python
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model
```

## セグメンテーション

### セマンティックセグメンテーションとインスタンスセグメンテーション

画素（pixel）単位で『分類』を行いたい場合に、『』を使用する。<br>
以下にセマンティック・セグメンテーションを示す。
![](https://cpp-learning.com/wp-content/uploads/2018/08/PSPNet-768x825.jpg)
上が入力画像、下が処理後の結果画像である。画素（pixel）１つ１つを分類し色付けすることができる。<br>
『セマンティック・セグメンテーション』の意味は以下の通りである。<br>

- セマンティック：情報を解釈する/意味付けする
- セグメンテーション：分類する/小さい単位に分ける
- セマンティック・セグメンテーション：画素（単位）で意味付けする/分類する

### インスタンス・セグメンテーション
『セマンティック・セグメンテーション』は画素（pixel）単位で分類が可能だが、同じラベルの物体が重なっている場合、物体同士の境界がわからなくなる。『物体検出』のように物体１つ１つの位置を検出し、かつ物体を画素（pixel）単位で『分類』したい場合、『インスタンス・セグメンテーション』を使用する。
![](https://cpp-learning.com/wp-content/uploads/2018/08/FCIS-768x488.jpg)

## 物体検出や画像分類との違い
### 画像分類
『画像分類』とは、画像に移っている物体が、事前に学習した物体のどれに一番近いかを分類する処理のことである。


【一致度リスト】

    フクロウ：99%
    鳥：95%
    猫：77%
    犬：42%
    靴：90%
    人：86%


![](https://cpp-learning.com/wp-content/uploads/2018/08/kururu500.jpg)

”フクロウの画像”を事前に学習していれば、フクロウとの一致率が一番高く出るが、フクロウ・はやぶさ・インコ・ニワトリなどを一括りで”鳥の画像”として学習させた場合は、鳥との一致度が一番高く出る。

このように学習させるデータ（訓練データ）と訓練データの分類（ラベル付け）の仕方によって結果が変わる。

### 物体検出
上記した『画像分類』では、物体が画像上のどの位置にいるかまでは判断できない。物体の位置まで把握したい場合は、Yoloなどの物体検出アルゴリズムを採用する。
![](https://cpp-learning.com/wp-content/uploads/2018/08/Object_Detect_Yolo.jpg)


物体の位置を検出できるため、バウンディングボックス（アンカーボックスとも呼ぶ）で物体を囲うことができる。また、物体の分類（ラベル名）と一致度も算出される。

### 物体検出や画像分類との違い
上記した『物体検出』には様々な手法があるが、良くも悪くも物体の位置を大まかにしか検出できない。
『セマンティック・セグメンテーション』を使用すれば、画素（pixel）単位で『分類』を行うことができる。

## セグメンテーション手法
### Fully Convolutional Networks for Semantic Segmentation
この論文の手法の最も重要な提案手法は、CNNの最後の全結合層をConvolutionに置き換えるということである。例えば、AlexNetのConv5の次の全結合層は6x6のConvolution層と見なすことができる。このようにCNNの全結合層をConvolution層に置き換えることで出力を分類クラスではなく二次元マップに変えることができる。
![](https://cdn-ak.f.st-hatena.com/images/fotolife/d/dl-kento/20170430/20170430215326.png)

### SegNet
風景画像のセグメンテーションを高速、省メモリで行う、というのが研究のモチベーションとなっている。
この手法の工夫の一つはネットワーク構造にある。FCNのネットワークの、全結合層を畳み込み層に置き換えた部分は計算量が多くなっていて学習に時間がかかったりメモリを多くとる原因となっていた。SegNetでは全結合層だった部分を削除したEncoder-Decoder構造を採用している。これによって高速化、省メモリ化を達成できた。
![](https://cdn-ak.f.st-hatena.com/images/fotolife/d/dl-kento/20170430/20170430224153.png)
SegNetの工夫は右半分のDecoder部分にもある。Decoder部分のMaxPoolingで採用した値の場所を記録し、upsampling時にその場所を使って特徴マップを拡大する。これにより位置情報を正確化させている。
![](https://cdn-ak.f.st-hatena.com/images/fotolife/d/dl-kento/20170430/20170430224454.png)

### U-net
U-netは医療画像のセグメンテーションのために提案されました 。セグメンテーションのタスクにおいて現時点でのstate-of-the-artと言われていてKaggleコンテストの優勝者が使っていた。

U-netではSegNetのようなEncoder-Decoder構造をしていて、Encoder部分とDecoder部分の対応した解像度の特徴マップをつないでいる。論文では図がU型に配置されていてこれがU-netの名前の由来。
![](https://cdn-ak.f.st-hatena.com/images/fotolife/d/dl-kento/20170430/20170430225135.png)
その他の工夫としては、重み付けロスの採用がある。
医療画像のセグメンテーションにおいてオブジェクト同士の境界をはっきり出すことは重要である。そのため、境界部分のロスを大きくし、はっきりと判読できるよう工夫している。つまり、タスクの目的にあった画像の前処理が必要ということである。

## スキップコネクション
### U-Netのようなセグメンテーション手法
U-netの場合のスキップコネクションはダウンサンプリングからアップサンプリングする際に、位置情報補完するために、ダウンサンプリング時の画像を「Copy and Corp」し、その後Convを行っている。。これにより、位置情報を補完している。
### ResNet[2]などの画像分類手法
RetNetの場合はUnetは違い、Convするの前の画像をコピーするのではなく、そのまま加算をしている。ただし、ダウンサンプリング等を行って次元やサイズが合わない場合は、出力画像の次元、サイズに合わせてから加算を行う。